In [16]:


from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional, Tuple
import re
import json

# --- Toggle this to False and implement your provider call in call_llm() ---
SIMULATION = True


# ---------------------------------------------------------------------------
# Core LLM adapter (replace this stub with your real model call if desired)
# ---------------------------------------------------------------------------
def call_llm(prompt: str, system: Optional[str] = None, temperature: float = 0.2) -> str:
    """
    Replace this with your real LLM SDK call. Keep the return as a plain string.
    For practice, we simulate a response so the rest of the script runs offline.
    """
    if SIMULATION:
        # Lightweight mock behavior: produce short, plausible-looking text.
        if "Classify the user's issue" in prompt:
            return "intent: billing_issue\nmissing_fields: ['account_id']"
        if "Ask only ONE question" in prompt:
            return "Sure—what is your 8-digit account ID?"
        if "Propose a resolution" in prompt:
            return ("resolution_steps:\n"
                    "1) Verify account_id in CRM\n"
                    "2) Check last invoice and applied discounts\n"
                    "3) Issue partial refund if overcharge confirmed\n"
                    "tone: empathetic")
        if "Draft the final message" in prompt:
            return ("Hi there—thanks for reaching out. I can help with billing.\n"
                    "Could you share your 8-digit account ID so I can look into the recent charge?\n"
                    "Once I have it, I’ll verify your last invoice and correct any overcharge.")
        if "ReACT-Style Code Task" in prompt:
            return (
                "Plan:\n"
                "- Normalize string: lowercase, remove non-alphanumerics.\n"
                "- Compare to its reverse.\n"
                "Final Code:\n"
                "```python\n"
                "import re\n"
                "def is_palindrome(s: str) -> bool:\n"
                "    cleaned = re.sub(r'[^0-9a-zA-Z]', '', s).lower()\n"
                "    return cleaned == cleaned[::-1]\n"
                "\n"
                "def _tests():\n"
                "    cases = [\n"
                "        (\"Madam, I'm Adam\", True),\n"
                "        (\"A man, a plan, a canal: Panama!\", True),\n"
                "        (\"palindrome\", False),\n"
                "        (\"No lemon, no melon\", True),\n"
                "    ]\n"
                "    for text, want in cases:\n"
                "        got = is_palindrome(text)\n"
                "        print(f'{text!r} -> {got} (want {want})')\n"
                "        assert got == want\n"
                "    return 'All tests passed.'\n"
                "\n"
                "if __name__ == '__main__':\n"
                "    print(_tests())\n"
                "```\n"
            )
        if "Critique the summary" in prompt:
            return ("Critique:\n"
                    "- Strengths: Clear, concise, captures main benefits.\n"
                    "- Gaps: No concrete metrics or examples; tone could be more customer-oriented.\n"
                    "- Improvements: Add a specific example and a brief call-to-action.\n")
        if "Rewrite an improved summary" in prompt:
            return ("Improved Summary:\n"
                    "Our new onboarding cuts setup from days to hours, guiding teams step-by-step and reducing errors.\n"
                    "For example, Acme Co. activated 120 users in one afternoon with <2% support tickets.\n"
                    "Start with the checklist, invite your team, and track progress from the dashboard.\n")
        # Default mock
        return "This is a mock response. Replace call_llm() to get real outputs."
    else:
        # Example (pseudocode) for real provider call; replace with your SDK:
        # from openai import OpenAI
        # client = OpenAI()
        # rsp = client.chat.completions.create(
        #     model="gpt-4o-mini",
        #     messages=[
        #         {"role": "system", "content": system or "You are a helpful assistant."},
        #         {"role": "user", "content": prompt},
        #     ],
        #     temperature=temperature,
        # )
        # return rsp.choices[0].message.content
        raise NotImplementedError("Implement your provider call here, or run in SIMULATION mode.")


# ---------------------------------------------------------------------------
# 1) Prompt Chaining for a Customer Support AI
# ---------------------------------------------------------------------------
@dataclass
class SupportState:
    user_message: str
    intent: Optional[str] = None
    missing_fields: List[str] = field(default_factory=list)
    collected: Dict[str, Any] = field(default_factory=dict)
    resolution_plan: Optional[str] = None
    tone: str = "friendly, concise, empathetic"


def chain_classify_issue(state: SupportState) -> SupportState:
    prompt = f"""
You are a customer support triage assistant.

Task: Classify the user's issue and list any missing fields required to proceed.

Constraints:
- Output in two lines only:
  - 'intent: <one_of>[billing_issue|technical_issue|account_access|other]'
  - 'missing_fields: [<comma-separated field names>]'

User said: {state.user_message}

Classify the user's issue and list missing fields.
"""
    print("\n[CHAIN STEP] Classification prompt:\n", prompt.strip(), "\n")
    raw = call_llm(prompt)
    print("[CHAIN STEP] Classification raw response:\n", raw, "\n")

    # Parse simple mock format
    intent_match = re.search(r"intent:\s*([a-z_]+)", raw)
    state.intent = intent_match.group(1) if intent_match else "other"

    missing_match = re.search(r"missing_fields:\s*\[(.*?)\]", raw, re.S)
    if missing_match:
        fields = [f.strip(" '\"\n") for f in missing_match.group(1).split(",") if f.strip()]
        state.missing_fields = fields

    return state


def chain_request_info(state: SupportState) -> Tuple[SupportState, str]:
    need = [f for f in state.missing_fields if f not in state.collected]
    if not need:
        return state, ""

    prompt = f"""
You are a support agent. Ask only ONE question that will help resolve the issue.
Be polite, brief, and ask for exactly this field: {need[0]}.
Return only the question, no extra text.
"""
    print("\n[CHAIN STEP] Request-info prompt:\n", prompt.strip(), "\n")
    question = call_llm(prompt)
    print("[CHAIN STEP] Request-info response:\n", question, "\n")
    return state, question.strip()


def chain_propose_resolution(state: SupportState) -> SupportState:
    prompt = f"""
You are an internal support playbook generator.

Context:
- Intent: {state.intent}
- Known fields: {json.dumps(state.collected)}
- Tone: {state.tone}

Task: Propose a resolution plan as numbered steps and set a tone label.

Output format (YAML-like):
resolution_steps:
  1) <step>
  2) <step>
tone: <one_or_two_words>
"""
    print("\n[CHAIN STEP] Resolution-plan prompt:\n", prompt.strip(), "\n")
    raw = call_llm(prompt)
    print("[CHAIN STEP] Resolution-plan response:\n", raw, "\n")
    state.resolution_plan = raw
    # Try to extract a tone hint
    m = re.search(r"tone:\s*([a-zA-Z _-]+)", raw)
    if m:
        state.tone = m.group(1).strip()
    return state


def chain_draft_final_message(state: SupportState) -> str:
    prompt = f"""
You are a frontline customer support agent.

Write the final reply to the user using this information:

- User message: {state.user_message}
- Intent: {state.intent}
- Collected fields: {json.dumps(state.collected)}
- Resolution plan:\n{state.resolution_plan}

Requirements:
- Keep it under 120 words.
- Use the tone: {state.tone}.
- If information is still missing (e.g., account_id), politely request it first.
- Be specific about next steps.
"""
    print("\n[CHAIN STEP] Final-message prompt:\n", prompt.strip(), "\n")
    msg = call_llm(prompt, temperature=0.4)
    print("[CHAIN STEP] Final-message response:\n", msg, "\n")
    return msg.strip()


def run_support_chain(example_user_message: str, collected: Optional[Dict[str, Any]] = None) -> str:
    state = SupportState(user_message=example_user_message, collected=collected or {})
    state = chain_classify_issue(state)
    state, question = chain_request_info(state)
    if question and not state.collected.get(state.missing_fields[0]):
        print("[ACTION NEEDED] Ask the user:", question)
        # In a real app, you'd wait for the user's answer here.
        # For demo, we pretend the user provided an id:
        state.collected[state.missing_fields[0]] = "12345678"

    state = chain_propose_resolution(state)
    return chain_draft_final_message(state)

if __name__ == "__main__":
    main()




[CHAIN STEP] Classification prompt:
 You are a customer support triage assistant.

Task: Classify the user's issue and list any missing fields required to proceed.

Constraints:
- Output in two lines only:
  - 'intent: <one_of>[billing_issue|technical_issue|account_access|other]'
  - 'missing_fields: [<comma-separated field names>]'

User said: I was charged twice on my last invoice.

Classify the user's issue and list missing fields. 

[CHAIN STEP] Classification raw response:
 intent: billing_issue
missing_fields: ['account_id'] 


[CHAIN STEP] Request-info prompt:
 You are a support agent. Ask only ONE question that will help resolve the issue.
Be polite, brief, and ask for exactly this field: account_id.
Return only the question, no extra text. 

[CHAIN STEP] Request-info response:
 Sure—what is your 8-digit account ID? 

Question: (SupportState(user_message='I was charged twice on my last invoice.', intent='billing_issue', missing_fields=['account_id'], collected={}, resolution_p